In [3]:
import GPy
import numpy as np
import sys
sys.path.append("../../")
from get_data import Data
import err

dim_set=[(0,),(1,),(2,),(0,1),(0,2),(1,2),(0,1,2,)]


for dim in dim_set:
    

    train_num_set=[52]


    kernel_set=[
                GPy.kern.Bias(input_dim=len(dim)),
                GPy.kern.Exponential(input_dim=len(dim)),
                GPy.kern.ExpQuad(input_dim=len(dim)),
                GPy.kern.GridRBF(input_dim=len(dim)),
                GPy.kern.Linear(input_dim=len(dim)),
                GPy.kern.Matern32(input_dim=len(dim)),
                GPy.kern.Matern52(input_dim=len(dim)),
                GPy.kern.MLP(input_dim=len(dim)),
                GPy.kern.OU(input_dim=len(dim)),
                GPy.kern.Poly(input_dim=len(dim)),
                GPy.kern.RBF(input_dim=len(dim)),
                GPy.kern.sde_Bias(input_dim=len(dim)),
                GPy.kern.sde_Exponential(input_dim=len(dim)),
                GPy.kern.sde_Matern32(input_dim=len(dim)),
                GPy.kern.sde_Matern52(input_dim=len(dim)),
                GPy.kern.sde_RatQuad(input_dim=len(dim)),
                GPy.kern.sde_RBF(input_dim=len(dim)),
                GPy.kern.sde_White(input_dim=len(dim)),
                GPy.kern.White(input_dim=len(dim))
                ]


    for kernel in kernel_set:
        for train_num in train_num_set:

            data=Data("../../data/data.txt",dim,train_num)

            data.train.mean_and_std()
            X=data.train.mean_and_std_X
            Y=data.train.mean_and_std_Y

            #构建高斯过程模型#
            
            #kernel=GPy.kern.Matern52(input_dim=len(dim))
            model=GPy.models.GPRegression(X,Y,kernel)
            #model.optimize(messages=True,max_f_eval = 1000)
            model.optimize(max_f_eval = 1000)


            # test 使用 train的数据归一化
            data.test.mean_and_std(data.train.mean_x,data.train.mean_y,
                                data.train.std_x,data.train.std_y)

            mean_train,var_train=model.predict(data.train.mean_and_std_X)
            #train 上的mean_train和var_train

            mean_test,var_test=model.predict(data.test.mean_and_std_X)
            # test 上的 mean_test 和 var_test

            #反归一化回原值
            data.train.re_mean_and_std(mean_train)
            data.test.re_mean_and_std(mean_test)



            tmp=""
            tmp2=""
            for a in dim:
                if(a==0):
                    tmp+="_0冠幅cw"
                    tmp2+="_0CW"
                elif(a==1):
                    tmp=tmp+"_1面积CA"
                    tmp2+="_1CA"
                elif(a==2):
                    tmp=tmp+"_2树高CH"
                    tmp2+="_2CH"

            
            err.show_err(
                (30,18),
                "../errbar_fig/"+str(dim)+type(kernel).__name__,
                
                data.train.predict_Y,
                data.train.Y,
                var_train,
                data.train.std_y[0],
                str("kernel: "+type(kernel).__name__)+
                    "\ninput_dim: "+str(tmp2)+
                    "\ntrain rmse: "+str(err.calc_rmse(data.train.Y,data.train.predict_Y))
                        ,
                data.test.predict_Y,
                data.test.Y,
                var_test,
                data.test.std_y[0],

                "\ntest rmse: "+str(err.calc_rmse(data.test.Y,data.test.predict_Y))+
                "\ntest Average error rate: "+str(err.calc_err_rate(data.test.Y,data.test.predict_Y))
                        
                )




 /home/he/anaconda3/envs/GPy_env/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning:More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
